In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import torch
from difflib import SequenceMatcher
from sentence_transformers import SentenceTransformer
device = "cuda" if torch.cuda.is_available() else "cpu"
args = {
    'data_set': 'acl_arc', 
    #'data_set' : 'sdp_act',
    'context_type': 'non_contiguous', 
    #'context_type': 'contiguous',
    'similarity': 0.4,
    'type': 'spaug'
}
MODEL = SentenceTransformer('../../model_training/output/multicite_fine_ref_2p_sppaug_1M_bi2024_04_26_15_24_15')

DATASET_DIR = f"../data/{args['data_set']}"
OUTPUT_DIR_PREV = os.path.join(DATASET_DIR, f"{args['data_set']}_dc_{'nc' if args['context_type']=='non_contiguous' else 'c'}_{args['exp_type']}_prev")
OUTPUT_DIR_NEXT = os.path.join(DATASET_DIR, f"{args['data_set']}_dc_{'nc' if args['context_type']=='non_contiguous' else 'c'}_{args['exp_type']}_next")
OUTPUT_DIR_COMB = os.path.join(DATASET_DIR, f"{args['data_set']}_dc_{'nc' if args['context_type']=='non_contiguous' else 'c'}_{args['exp_type']}_comb")

for OUTPUT_DIR in [OUTPUT_DIR_PREV, OUTPUT_DIR_NEXT, OUTPUT_DIR_COMB]:
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

In [ ]:
def extract_dynamic_context (data_df, context_type, similarity):
    dynamic_contexts_prev = []
    dynamic_contexts_next = []
    dynamic_contexts_combined = []
    
    data_df['cite_context_paragraph'] = data_df['cite_context_paragraph'].apply(eval)
    for idx, row in data_df.iterrows():
        paragraph = row['cite_context_paragraph']
        citation_context = row['citation_context']

        if context_type== 'non_contiguous':         
            dynamic_context_prev, dynamic_context_next, dynamic_context_combined = extract_non_contigious_context(citation_context, paragraph)
            
        else:
            dynamic_context_prev, dynamic_context_next, dynamic_context_combined = extract_contigious_context(citation_context, paragraph)
            
        dynamic_contexts_next.append(dynamic_context_next)
        dynamic_contexts_prev.append(dynamic_context_prev)
        dynamic_contexts_combined.append(dynamic_context_combined)

    return dynamic_contexts_prev, dynamic_contexts_next, dynamic_contexts_combined 

def extract_contigious_context ():
    dynamic_context_prev = []
    dynamic_context_next = []
    dynamic_context_combined = []
    
    
    
    return dynamic_context_prev, dynamic_context_next, dynamic_context_combined

def extract_non_contigious_context():
    return
    

In [ ]:
for dataset in ["train", "test"]:
    print(dataset)
    data_df = pd.read_csv(DATASET_DIR + f"/{dataset}_raw.txt", sep="\t", engine="python", dtype=object)
    dynamic_contexts_prev, dynamic_contexts_next, dynamic_contexts_combined = \
        extract_dynamic_context(data_df, args['context_type'], args['exp_type'])
    prev_df = pd.DataFrame({'CC': dynamic_contexts_prev,
                            'label': data_df['citation_class_label']})
    next_df = pd.DataFrame({'CC': dynamic_contexts_next,
                            'label': data_df['citation_class_label']})
    comb_df = pd.DataFrame({'CC': dynamic_contexts_combined,
                            'label': data_df['citation_class_label']})   
    
    if dataset == 'train':
        prev_df.to_csv(OUTPUT_DIR_PREV + '/train.csv', index=False)
        next_df.to_csv(OUTPUT_DIR_NEXT + '/train.csv', index=False)
        comb_df.to_csv(OUTPUT_DIR_COMB + '/train.csv', index=False)

    else:
        prev_df.to_csv(OUTPUT_DIR_PREV + '/test.csv', index=False)
        next_df.to_csv(OUTPUT_DIR_NEXT + '/test.csv', index=False)
        comb_df.to_csv(OUTPUT_DIR_COMB + '/test.csv', index=False)
